# Импорты


In [477]:
import numpy as np

In [478]:
def calc_lambda(A: np.matrix, b: np.matrix) -> np.matrix:
    return float((b.T @ A @ b) / (b.T @ b))

In [479]:
def power_iteration(
    A: np.matrix, x_k=None, max_iterations: int = 100000, tolerance: float = 1e-3
) -> np.matrix:
    # print("POWER:\n")
    # Ideally choose a random vector
    # To decrease the chance that our vector
    # Is orthogonal to the eigenvector
    if x_k is None:
        x_k = np.random.rand(A.shape[1], 1)
    # print(b_k)

    lambda_prev = None
    
    iter = 1

    for _ in range(max_iterations):
        x_k1 = A @ x_k
        x_k1 = x_k1 / np.linalg.norm(x_k1)

        lambda_current = calc_lambda(A, x_k1)
        # lambda_current = np.max(np.abs(x_k1[:-1]/x_k[:-1]))

        if lambda_prev is not None and abs(lambda_current - lambda_prev) <= tolerance:
            break

        lambda_prev = lambda_current
        x_k = x_k1
        iter += 1

    # print("\n", b_k)

    return x_k1, lambda_current, iter

In [480]:
def jacobi_rotation(A, max_iterations=100000, tolerance=1e-7):
    n = A.shape[0]
    # Инициализация матрицы собственных векторов
    eigenvectors = np.eye(n)
    iter = 0

    for _ in range(max_iterations):
        iter += 1
        # Нахождение индексов максимального элемента вне главной диагонали
        off_diagonal = A.copy()
        np.fill_diagonal(off_diagonal, 0)
        i, j = np.unravel_index(np.argmax(np.abs(off_diagonal)), off_diagonal.shape)

        # Проверка на сходимость
        # print(np.sum(np.square(off_diagonal)))
        if np.sum(np.square(off_diagonal)) <= tolerance:
            break

        # Вычисление угла вращения
        if A[j, j] == A[i, i]:
            theta = np.pi / 4
        else:
            theta = 0.5 * np.arctan2(2 * A[i, j], A[i, i] - A[j, j])

        # Формирование вращающей матрицы
        c = np.cos(theta)
        s = np.sin(theta)

        J = np.eye(n)
        J[i, i] = c
        J[j, j] = c
        J[i, j] = -s
        J[j, i] = s

        # Обновление матрицы A и матрицы собственных векторов
        A = J.T @ A @ J
        eigenvectors = eigenvectors @ J

    eigenvalues = np.diagonal(A)
    return eigenvalues, eigenvectors, iter

$\begin{gathered}\lambda_{1}=7.1190267\\\lambda_{2}=2.5683729\\\lambda_{3}=1.3126004\end{gathered}$


In [481]:
def gen_matrix(size: int):
    A = np.random.randint(-256,256,size=(size, size))
    return A.T @ A

A3 = gen_matrix(3)
A5 = gen_matrix(5)
A7 = gen_matrix(7)
mass_A = [A3, A5, A7]

mass_A

[array([[ 97625,  71810,   8970],
        [ 71810,  60350,  36222],
        [  8970,  36222, 126054]], dtype=int32),
 array([[105215, -22088,  -7939,   2044, -69306],
        [-22088,  90731, -33769, -23374, -26027],
        [ -7939, -33769, 133089, -26201, 143838],
        [  2044, -23374, -26201,  77992, -38155],
        [-69306, -26027, 143838, -38155, 192793]], dtype=int32),
 array([[ 133306,  -71431,  -90382,   11265,   53726,  -16696,   12419],
        [ -71431,  211556,  -19470,  -34991, -105855,   66305,  -28484],
        [ -90382,  -19470,  102894,   22128,   16074,   26419,  -30306],
        [  11265,  -34991,   22128,  206699,   92889,   77768, -126574],
        [  53726, -105855,   16074,   92889,  184814,  -14948,  -14635],
        [ -16696,   66305,   26419,   77768,  -14948,  176146, -124937],
        [  12419,  -28484,  -30306, -126574,  -14635, -124937,  181808]],
       dtype=int32)]

In [482]:
for A in mass_A:
    print("Матрица:")
    print(A)
    for eps in [1e-3, 1e-7]:
        print("Точность:")
        print(eps)
        print()
        print("Якоби:")
        eigenvalues, eigenvectors, iters = jacobi_rotation(A, tolerance=eps)
        print("Собственные значения:")
        print(eigenvalues)
        print("Собственные векторы:")
        print(eigenvectors)
        print("Количество итераций:")
        print(iters)
        print()
        
        print("Прямые итерации:")
        eigenvectors, eigenvalues, iters = power_iteration(A)
        print("Собственное значение:")
        print(eigenvalues)
        print("Собственный вектор:")
        print(eigenvectors)
        print("Количество итераций:")
        print(iters)
    print()
    print()
         

Матрица:
[[ 97625  71810   8970]
 [ 71810  60350  36222]
 [  8970  36222 126054]]
Точность:
0.001

Якоби:
Собственные значения:
[   332.40822554 110874.71019274 172821.88158173]
Собственные векторы:
[[-0.57176757 -0.55099578  0.60785319]
 [ 0.79830005 -0.20278104  0.56709513]
 [-0.18920592  0.80949583  0.55580358]]
Количество итераций:
8

Прямые итерации:
Собственное значение:
172821.88097152088
Собственный вектор:
[[0.60779842]
 [0.56707497]
 [0.55588403]]
Количество итераций:
19
Точность:
1e-07

Якоби:
Собственные значения:
[   332.40822554 110874.71019274 172821.88158173]
Собственные векторы:
[[-0.57176757 -0.55099586  0.60785311]
 [ 0.79830005 -0.20278112  0.5670951 ]
 [-0.18920592  0.80949576  0.55580369]]
Количество итераций:
9

Прямые итерации:
Собственное значение:
172821.88125919795
Собственный вектор:
[[0.60781335]
 [0.56708047]
 [0.5558621 ]]
Количество итераций:
18


Матрица:
[[105215 -22088  -7939   2044 -69306]
 [-22088  90731 -33769 -23374 -26027]
 [ -7939 -33769 133089 

C:\Users\artem\AppData\Local\Temp\ipykernel_19416\2408308288.py:2: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float((b.T @ A @ b) / (b.T @ b))


Как видим, прямые итерации работают быстрее (примерно в 2 раза), но при этом не получаем все СЗ.

Так что нужно использовать эти методы в зависимости от того, что нам нужно